<a href="https://colab.research.google.com/github/sm6592624/project.pd/blob/main/Copy_of_temperature_forecast_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML-enhanced weather forecasting

This notebook demonstrates how to use machine-learning to enhance the output from a numerical weather prediction model.

In [1]:
import pandas as pd
import numpy as np
from plotly import graph_objects as go

In [ ]:
from google.colab import files
uploaded = files.upload()

## Load dataset

This dataset contains:
1.  Historical 24hr-ahead forecasts from [NOAA's GFS weather model](https://www.ncdc.noaa.gov/data-access/model-data/model-datasets/global-forcast-system-gfs); and
2.  Actual temperature data recorded at the [Max Planck Institute for Biogeochemistry in Jena, Germany](https://www.bgc-jena.mpg.de/wetter/).

Initially, we load the data and examine the forecast accuracy for 2019.

# New Section

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
creation=pd.read_csv('Book1.csv', sep=';')
creation.head(5)
print(creation)
print("The shape of our feature is:", creation.shape)
creation.describe()
creation=pd.get_dummies(creation)
creation.iloc[:,5:].head(5)
labels=np.array(creation['Temperature'])
creation=creation.drop('Temperature',axis=1)
creation_list=list(creation.columns)
creation=np.array(creation)

In [ ]:
train_creation, test_creation, train_labels, test_labels= train(creation,labels, test_size=0.30,random_state=4)

In [ ]:
# Plot actual and forecast for 2019
traces = [
          go.Scatter(x=df_test.index, y=df_test['actual'], mode='lines', name='Actual'),
          go.Scatter(x=df_test.index, y=df_test['surface_temperature_forecast'], mode='lines', name='Surface forecast'),
          go.Scatter(x=df_test.index, y=df_test['2m_temperature_forecast'], mode='lines', name='2m forecast'),
]
fig = go.Figure(
    data=traces,
    layout={
        'title': 'Jena temperature: actual and forecast',
        'yaxis': {'title': 'Temperature, °C'},
        'margin': {'l': 10, 'r': 10, 't': 25, 'b': 10}, 'template': 'plotly_white'
    },
)
fig

NameError: ignored

In [ ]:
# Plot histograms of forecast error
traces = [
          go.Histogram(x=df_test['surface_temperature_forecast'] - df_test['actual'], name='Error on surface forecast', histnorm='probability'),
          go.Histogram(x=df_test['2m_temperature_forecast'] - df_test['actual'], name='Error on 2m forecast', histnorm='probability'),
]
fig = go.Figure(
    data=traces,
    layout={
        'title': 'Forecast error',
        'xaxis': {'title': 'Error, °C'},
        'margin': {'l': 10, 'r': 10, 't': 25, 'b': 10}, 'template': 'plotly_white'
    },
)
fig

In [ ]:
# Calculate root-mean-squared (RMS) errors on forecasts
from sklearn.metrics import mean_squared_error

def rms_error(y_true, y_predicted):
  return mean_squared_error(y_true.flatten(), y_predicted.flatten()) ** 0.5

In [ ]:
print(f'RMS Error on surface forecast {rms_error(df_test["actual"].values, df_test["surface_temperature_forecast"].values):.2f} °C')
print(f'RMS Error on 2m height forecast {rms_error(df_test["actual"].values, df_test["2m_temperature_forecast"].values):.2f} °C')

RMS Error on surface forecast 2.88 °C
RMS Error on 2m height forecast 2.05 °C


## ML-enhanced forecast: linear model

Here we use a simple model (linear regression) to improve forecast accuracy for our specific forecasting problem. The model learns to improve the output of the GFS weather model as applied to the temperature measured in Jena. We use the time of day as an additional feature to help improve model performance.


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Feature engineering: add time as cyclically-encoded variables
df_train = df_train.assign(
    sine_time=np.sin(2 * np.pi * df_train.index.hour / 24),
    cosine_time=np.cos(2 * np.pi * df_train.index.hour / 24),
)
df_test = df_test.assign(
    sine_time=np.sin(2 * np.pi * df_test.index.hour / 24),
    cosine_time=np.cos(2 * np.pi * df_test.index.hour / 24),
)

In [ ]:
# Feature engineering: add forecast from previous timestep as feature
df_train = df_train.assign(
    lagged_surface=df_train['surface_temperature_forecast'].shift(1),
    lagged_2m=df_train['2m_temperature_forecast'].shift(1),
).dropna()
df_test = df_test.assign(
    lagged_surface=df_test['surface_temperature_forecast'].shift(1),
    lagged_2m=df_test['2m_temperature_forecast'].shift(1),
).dropna()

In [ ]:
# Use GFS forecasts and engineered variables to predict actual temperature measurement
X_vars = df_train.columns.difference(['actual'])
y_vars = ['actual']

X_train = df_train[X_vars].values
y_train = df_train[y_vars].values
X_test = df_test[X_vars].values
y_test = df_test[y_vars].values

In [ ]:
model_lr = LinearRegression()
model_lr.fit(X=X_train, y=y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_predict_lr = model_lr.predict(X_test)
print(f'RMS Error on linear model forecast {rms_error(df_test["actual"].values, y_predict_lr):.2f} °C')

RMS Error on linear model forecast 1.77 °C


## Summary of results

In [ ]:
# Plot histograms of forecast error
traces = [
          go.Histogram(x=df_test['surface_temperature_forecast'] - df_test['actual'], name='Error on surface forecast', histnorm='probability'),
          go.Histogram(x=df_test['2m_temperature_forecast'] - df_test['actual'], name='Error on 2m forecast', histnorm='probability'),
          go.Histogram(x=y_predict_lr.flatten() - df_test['actual'], name='Error on ML-enhanced forecast', histnorm='probability'),
]
fig = go.Figure(
    data=traces,
    layout={
        'title': 'Forecast error',
        'xaxis': {'title': 'Error, °C'},
        'margin': {'l': 10, 'r': 10, 't': 25, 'b': 10}, 'template': 'plotly_white'
    },
)
fig

In [ ]:
# Bar chart of RMS errors
traces = [
          go.Bar(
              x=['Surface forecast', '2m height forecast', 'ML-enhanced forecast'],
              y=[
                 rms_error(df_test['actual'].values, df_test['surface_temperature_forecast'].values),
                 rms_error(df_test['actual'].values, df_test['2m_temperature_forecast'].values),
                 rms_error(df_test['actual'].values, y_predict_lr),
              ],
          ),
]
fig = go.Figure(
    data=traces,
    layout={
        'title': 'Forecast RMS error on each model',
        'yaxis': {'title': 'RMS Error, °C'},
        'margin': {'l': 10, 'r': 10, 't': 25, 'b': 10}, 'template': 'plotly_white'
    },
)
fig